In [1]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.models.model import *
from pathlib import Path
import numpy as np

In [2]:
root = Path('/data2/project_model_based_fmri/ds001882/')
save_path = Path('scratch')
mask_path = root / 'derivatives' / 'fmriprep' / 'mask'

## PREPROCESS FMRI IMAGE

In [3]:
X, voxel_mask, layout = bids_preprocess(root,mask_path=mask_path,save_path=save_path,smoothing_fwhm=None, zoom=(2, 2, 2),ncore=2, nthread=2,)

image preprocessing - fMRI data..                 :  67%|██████▋   | 4/6 [00:45<00:40, 20.12s/it]

ValueError: range() arg 3 must not be zero

## PREPRROCESS EVENT DATA

In [ ]:
def example_prep_func_piva_dd(row,info):

    
    ## user defined mapping ##
    if row['delay_left'] >= row['delay_right']:
        row['delay_later'] = row['delay_left']
        row['delay_sooner'] = row['delay_right']
        row['amount_later'] = row['money_left']
        row['amount_sooner'] = row['money_right']
        row['choice'] = 1 if row['choice'] == 1 else 0
    else:
        row['delay_later'] = row['delay_right']
        row['delay_sooner'] = row['delay_left']
        row['amount_later'] = row['money_right']
        row['amount_sooner'] = row['money_left']
        row['choice'] = 1 if row['choice'] == 2 else 0
        
    return row


def example_cond_func_piva_dd(row):
    return row['agent']==0

dm_model = 'dd_hyperbolic'

def example_latent_func_piva_dd(row,param_dict):
    
    ev_later   = row['amount_later'] / (1 + param_dict['k'] * row['delay_later'])
    ev_sooner  = row['amount_sooner'] / (1 + param_dict['k'] * row['delay_sooner'])
    utility = ev_later - ev_sooner
    row['modulation'] = utility
    
    return row

In [ ]:
dm_model = 'dd_hyperbolic'

In [ ]:
df_events = pd.read_csv( 'df_events.tsv',sep = '\t',index_col='Unnamed: 0')

In [10]:
all_individual_params = None #pd.read_csv('all_ind_pars.tsv',sep = '\t',index_col='Unnamed: 0')

In [11]:
dm_model, df_events, signals, time_masks = preprocess_events(root, 
                                          hrf_model="glover",
                                          normalizer="minmax",
                                          dm_model=dm_model,
                                          latent_func=example_latent_func_piva_dd, 
                                          layout=None,
                                          preprocess=example_prep_func_piva_dd,
                                          condition=example_cond_func_piva_dd,
                                          df_events=df_events,
                                          all_individual_params=all_individual_params,
                                          use_duration=False,
                                          save=True,
                                          save_path=save_path,
                                          ncore=16
                                          )



  0%|          | 0/6 [00:00<?, ?it/s]
loading bids dataset..                            :   0%|          | 0/6 [00:00<?, ?it/s]
loading bids dataset..                            :  17%|█▋        | 1/6 [00:20<01:43, 20.75s/it]
adjusting event file columns..                    :  17%|█▋        | 1/6 [00:20<01:43, 20.75s/it]
adjusting event file columns..                    :  33%|███▎      | 2/6 [00:29<01:08, 17.09s/it]
calculating time mask..                           :  33%|███▎      | 2/6 [00:29<01:08, 17.09s/it]/home/cheoljun/anaconda3/envs/model_based_fmri/lib/python3.7/site-packages/model_based_mvpa-0.0.1-py3.7.egg/model_based_mvpa/preprocessing/events.py:318: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  time_masks = np.array(time_masks)

calculating time mask.. 

## LOAD DATA

In [3]:
prep_path_X = Path('.')
prep_path_y = Path('/home/cheoljun/project_model_based_fmri/examples/Delayed-discount_task/scratch')
masked_data_path = Path('.')

In [4]:
X, y = prepare_data(prep_path_X=prep_path_X, prep_path_y=prep_path_y)

In [5]:
print(X.shape, y.shape)

(25314, 5066) (25314,)


In [6]:
masked_data = get_mask_image(masked_data_path=masked_data_path) 

## FIT MVPA MODEL

In [7]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [8]:
print(time.strftime('%c', time.localtime(time.time())))
N = 30
coefs = mlp_regression(X, y,
                       layer_dims=[1024, 1024],
                       activation='linear',
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=30,
                       verbose=1,
                       save_path = '.')

task_name = f'piva2019_mlp_{N}'
result = get_map(coefs, masked_data, task_name=task_name, map_type='z', save_path='.', smoothing_sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

INFO:root:start running


Mon Nov 23 15:55:38 2020


INFO:root:[1/30] - mse: 1.5037
INFO:root:[2/30] - mse: 1.4430


KeyboardInterrupt: 

In [9]:
import random

In [23]:
f'{int(random.random()*100000)}'

'14981'

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))
N=1
alpha  = 0.005
coefs = elasticnet(X, y,
               alpha=alpha,
               n_splits=5,
               n_jobs=16,
               max_lambda = 10,
               min_lambda_ratio = 1e-4,
               lambda_search_num = 200,
               N=N,
               verbose=1,
               max_use_sample_N=30000)

task_name = f'piva2019_elasticnet_{N}_{alpha:.4f}'
result = get_map(coefs, masked_data, task_name=task_name, map_type='z', save_path='.', smoothing_sigma=1)
print(time.strftime('%c', time.localtime(time.time())))